<a href="https://colab.research.google.com/github/samxc/MachineLearningAndAI/blob/main/Pytorch_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26421880/26421880 [00:01<00:00, 16142589.76it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 271090.02it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 5091696.54it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 19864100.27it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [ ]:
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [ ]:
batch_size = 64
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader and train_dataloader:
    print(f"Shape of X[N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape}{y.dtype}")
    break

Shape of X[N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64])torch.int64


In [ ]:
#Get cpu, gpu or mps device for training
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        #compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        #Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}[{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg_loss: {test_loss:>8f}\n")

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t + 1}\n----------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
----------------------------
loss: 2.176505[   64/60000]
loss: 2.156713[ 6464/60000]
loss: 2.109294[12864/60000]
loss: 2.111561[19264/60000]
loss: 2.065493[25664/60000]
loss: 2.012950[32064/60000]
loss: 2.036923[38464/60000]
loss: 1.962946[44864/60000]
loss: 1.963386[51264/60000]
loss: 1.883764[57664/60000]
Test Error: 
 Accuracy: 58.1%, Avg_loss: 1.888625

Epoch 2
----------------------------
loss: 1.933441[   64/60000]
loss: 1.887275[ 6464/60000]
loss: 1.785888[12864/60000]
loss: 1.807364[19264/60000]
loss: 1.699779[25664/60000]
loss: 1.662426[32064/60000]
loss: 1.673888[38464/60000]
loss: 1.583735[44864/60000]
loss: 1.605296[51264/60000]
loss: 1.487065[57664/60000]
Test Error: 
 Accuracy: 61.6%, Avg_loss: 1.511682

Epoch 3
----------------------------
loss: 1.593840[   64/60000]
loss: 1.536158[ 6464/60000]
loss: 1.401610[12864/60000]
loss: 1.453547[19264/60000]
loss: 1.337019[25664/60000]
loss: 1.346834[32064/60000]
loss: 1.351560[38464/60000]
loss: 1.282341[44864/60000]
los

In [ ]:
#Saving the model by serializing the internal state dictionary(
    #containing the model parameters)
torch.save(model.state_dict(), "model.pth")
print("saved pyTorch Model state to model.pth")

saved pyTorch Model state to model.pth


In [ ]:
#Loading Models
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [ ]:
#Making prediction using the loaded model

classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot"
]

model.eval()
x, y = test_data[1][0], test_data[1][1]
print(f"x-data is: {x} and y-data is: {y}")
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

x-data is: tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0510, 0.2627, 0.0000, 0.0000, 0.0000, 0.0000,
          0.1961, 0.1490, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0314,
          0.4706, 0.8196, 0.8863, 0.9686, 0.9294, 1.0000, 1.0000, 1.0000,
          0.9686, 0.9333, 0.9216, 0.6745, 0.2824, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5373, 0.9373,
          0.9882, 0.9529, 0.9176, 0.8980, 0.9333, 0.9569, 0.9647, 0.9412,
          0.9020, 0.9098, 0.9373, 0.9725, 0.9843, 0.7608, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.4000, 1.0000, 0.9059,
          0.8941, 0.8902, 0.8941, 0.9137, 0.9020, 0.9020, 0.8980, 0.8941,
          0.9098, 0.9098, 0.9059, 0.8902, 0.87